<center>
<img src="https://habrastorage.org/web/677/8e1/337/6778e1337c3d4b159d7e99df94227cb2.jpg"/>
## Специализация "Машинное обучение и анализ данных"
<center>Автор материала: программист-исследователь Mail.Ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ [Юрий Кашницкий](https://yorko.github.io/)

# <center> Capstone проект №1 <br> Идентификация пользователей по посещенным веб-страницам
<img src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

# <center>Неделя 5.  Соревнование Kaggle "Catch Me If You Can"

На этой неделе мы вспомним про концепцию стохастического градиентного спуска и опробуем классификатор Scikit-learn SGDClassifier, который работает намного быстрее на больших выборках, чем алгоритмы, которые мы тестировали на 4 неделе. Также мы познакомимся с данными [соревнования](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) Kaggle по идентификации пользователей и сделаем в нем первые посылки. По итогам этой недели дополнительные баллы получат те, кто попадет в топ-30 публичного лидерборда соревнования.

**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций курса "Обучение на размеченных данных":**
   - [Стохатический градиентный спуск](https://www.coursera.org/learn/supervised-learning/lecture/xRY50/stokhastichieskii-ghradiientnyi-spusk)
   - [Линейные модели. Sklearn.linear_model. Классификация](https://www.coursera.org/learn/supervised-learning/lecture/EBg9t/linieinyie-modieli-sklearn-linear-model-klassifikatsiia)
   
**Также рекомендуется вернуться и просмотреть [задание](https://www.coursera.org/learn/supervised-learning/programming/t2Idc/linieinaia-rieghriessiia-i-stokhastichieskii-ghradiientnyi-spusk) "Линейная регрессия и стохастический градиентный спуск" 1 недели 2 курса специализации.**

### Задание
1. Заполните код в этой тетрадке 
2. Если вы проходите специализацию Яндеса и МФТИ, пошлите тетрадку в соответствующем Peer Review. <br> Если вы проходите курс ODS, выберите ответы в [веб-форме](https://docs.google.com/forms/d/1pLsegkAICL9PzOLyAeH9DmDOBfktte0l8JW75uWcTng). 

In [490]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import os
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV

**Считаем данные [соревнования](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) в DataFrame train_df и test_df (обучающая и тестовая выборки).**

In [2]:
# Поменяйте на свой путь к данным
PATH_TO_DATA = 'capstone_user_identification'

In [3]:
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'),
                       index_col='session_id')
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'),
                      index_col='session_id')

In [4]:
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0


**Объединим обучающую и тестовую выборки – это понадобится, чтоб вместе потом привести их к разреженному формату.**

In [5]:
train_test_df = pd.concat([train_df, test_df])

В обучающей выборке видим следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - user_id – ID пользователя
    
Сессии пользователей выделены таким образом, что они не могут быть длинее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд, либо когда сессия заняла по времени более 30 минут. 

**Посмотрим на статистику признаков.**

Пропуски возникают там, где сессии короткие (менее 10 сайтов). Скажем, если человек 1 января 2015 года посетил *vk.com* в 20:01, потом *yandex.ru* в 20:29, затем *google.com* в 20:33, то первая его сессия будет состоять только из двух сайтов (site1 – ID сайта *vk.com*, time1 – 2015-01-01 20:01:00, site2 – ID сайта  *yandex.ru*, time2 – 2015-01-01 20:29:00, остальные признаки – NaN), а начиная с *google.com* пойдет новая сессия, потому что уже прошло более 30 минут с момента посещения *vk.com*.

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 1 to 253561
Data columns (total 21 columns):
site1     253561 non-null int64
time1     253561 non-null object
site2     250098 non-null float64
time2     250098 non-null object
site3     246919 non-null float64
time3     246919 non-null object
site4     244321 non-null float64
time4     244321 non-null object
site5     241829 non-null float64
time5     241829 non-null object
site6     239495 non-null float64
time6     239495 non-null object
site7     237297 non-null float64
time7     237297 non-null object
site8     235224 non-null float64
time8     235224 non-null object
site9     233084 non-null float64
time9     233084 non-null object
site10    231052 non-null float64
time10    231052 non-null object
target    253561 non-null int64
dtypes: float64(9), int64(2), object(10)
memory usage: 42.6+ MB


In [7]:
test_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,29,2014-10-04 11:19:53,35.0,2014-10-04 11:19:53,22.0,2014-10-04 11:19:54,321.0,2014-10-04 11:19:54,23.0,2014-10-04 11:19:54,2211.0,2014-10-04 11:19:54,6730.0,2014-10-04 11:19:54,21.0,2014-10-04 11:19:54,44582.0,2014-10-04 11:20:00,15336.0,2014-10-04 11:20:00
2,782,2014-07-03 11:00:28,782.0,2014-07-03 11:00:53,782.0,2014-07-03 11:00:58,782.0,2014-07-03 11:01:06,782.0,2014-07-03 11:01:09,782.0,2014-07-03 11:01:10,782.0,2014-07-03 11:01:23,782.0,2014-07-03 11:01:29,782.0,2014-07-03 11:01:30,782.0,2014-07-03 11:01:53
3,55,2014-12-05 15:55:12,55.0,2014-12-05 15:55:13,55.0,2014-12-05 15:55:14,55.0,2014-12-05 15:56:15,55.0,2014-12-05 15:56:16,55.0,2014-12-05 15:56:17,55.0,2014-12-05 15:56:18,55.0,2014-12-05 15:56:19,1445.0,2014-12-05 15:56:33,1445.0,2014-12-05 15:56:36
4,1023,2014-11-04 10:03:19,1022.0,2014-11-04 10:03:19,50.0,2014-11-04 10:03:20,222.0,2014-11-04 10:03:21,202.0,2014-11-04 10:03:21,3374.0,2014-11-04 10:03:22,50.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:23,3374.0,2014-11-04 10:03:23
5,301,2014-05-16 15:05:31,301.0,2014-05-16 15:05:32,301.0,2014-05-16 15:05:33,66.0,2014-05-16 15:05:39,67.0,2014-05-16 15:05:40,69.0,2014-05-16 15:05:40,70.0,2014-05-16 15:05:40,68.0,2014-05-16 15:05:40,71.0,2014-05-16 15:05:40,167.0,2014-05-16 15:05:44


In [8]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82797 entries, 1 to 82797
Data columns (total 20 columns):
site1     82797 non-null int64
time1     82797 non-null object
site2     81308 non-null float64
time2     81308 non-null object
site3     80075 non-null float64
time3     80075 non-null object
site4     79182 non-null float64
time4     79182 non-null object
site5     78341 non-null float64
time5     78341 non-null object
site6     77566 non-null float64
time6     77566 non-null object
site7     76840 non-null float64
time7     76840 non-null object
site8     76151 non-null float64
time8     76151 non-null object
site9     75484 non-null float64
time9     75484 non-null object
site10    74806 non-null float64
time10    74806 non-null object
dtypes: float64(9), int64(1), object(10)
memory usage: 13.3+ MB


**В обучающей выборке – 2297 сессий одного пользователя (Alice) и 251264 сессий – других пользователей, не Элис. Дисбаланс классов очень сильный, и смотреть на долю верных ответов (accuracy) непоказательно.**

In [9]:
train_df['target'].value_counts()

0    251264
1      2297
Name: target, dtype: int64

**Пока для прогноза будем использовать только индексы посещенных сайтов. Индексы нумеровались с 1, так что заменим пропуски на нули.**

In [10]:
train_test_df_sites = train_test_df[['site%d' % i for i in range(1, 11)]].fillna(0).astype('int')

In [11]:
train_test_df_sites.head(10)

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,718,0,0,0,0,0,0,0,0,0
2,890,941,3847,941,942,3846,3847,3846,1516,1518
3,14769,39,14768,14769,37,39,14768,14768,14768,14768
4,782,782,782,782,782,782,782,782,782,782
5,22,177,175,178,177,178,175,177,177,178
6,570,21,570,21,21,0,0,0,0,0
7,803,23,5956,17513,37,21,803,17514,17514,17514
8,22,21,29,5041,14422,23,21,5041,14421,14421
9,668,940,942,941,941,942,940,23,21,22


**Создайте разреженные матрицы *X_train_sparse* и *X_test_sparse* аналогично тому, как мы это делали ранее. Используйте объединенную матрицу *train_test_df_sites*, потом разделите обратно на обучающую и тестовую части.**

Обратите внимание на то, что в  сессиях меньше 10 сайтов  у нас остались нули, так что первый признак (сколько раз попался 0) по смыслу отличен от остальных (сколько раз попался сайт с индексом $i$). Поэтому первый столбец разреженной матрицы надо будет удалить.

**Выделите в отдельный вектор *y* ответы на обучающей выборке.**

In [12]:
train_df[['time' + str(i) for i in range(1, 11)]][:5]

,time1,time2,time3,time4,time5,time6,time7,time8,time9,time10
session_id,,,,,,,,,,
1,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-02-22 11:19:50,2014-02-22 11:19:50,2014-02-22 11:19:51,2014-02-22 11:19:51,2014-02-22 11:19:51,2014-02-22 11:19:51,2014-02-22 11:19:52,2014-02-22 11:19:52,2014-02-22 11:20:15,2014-02-22 11:20:16
3,2013-12-16 16:40:17,2013-12-16 16:40:18,2013-12-16 16:40:19,2013-12-16 16:40:19,2013-12-16 16:40:19,2013-12-16 16:40:19,2013-12-16 16:40:20,2013-12-16 16:40:21,2013-12-16 16:40:22,2013-12-16 16:40:24
4,2014-03-28 10:52:12,2014-03-28 10:52:42,2014-03-28 10:53:12,2014-03-28 10:53:42,2014-03-28 10:54:12,2014-03-28 10:54:42,2014-03-28 10:55:12,2014-03-28 10:55:42,2014-03-28 10:56:12,2014-03-28 10:56:42
5,2014-02-28 10:53:05,2014-02-28 10:55:22,2014-02-28 10:55:22,2014-02-28 10:55:23,2014-02-28 10:55:23,2014-02-28 10:55:59,2014-02-28 10:55:59,2014-02-28 10:55:59,2014-02-28 10:57:06,2014-02-28 10:57:11


In [13]:
data_without_user_id = train_test_df_sites.iloc[:, 0:10]
flatten = data_without_user_id.values.flatten()
data_csr = [1] * len(flatten)
indptr_csr = np.arange(0, len(flatten) + 1, 10)
X_sparse = csr_matrix((data_csr, flatten, indptr_csr))[:, 1:]
y_sparse = train_test_df['target']

train_test_sparse = X_sparse#''' ВАШ КОД ЗДЕСЬ '''
X_train_sparse = X_sparse[:253561]#''' ВАШ КОД ЗДЕСЬ '''
X_test_sparse = X_sparse[253561:]#''' ВАШ КОД ЗДЕСЬ '''
y = y_sparse[:253561]#''' ВАШ КОД ЗДЕСЬ '''

In [14]:
X_sparse.shape

(336358, 48371)

In [15]:
train_test_df_sites.shape

(336358, 10)

In [16]:
X_test_sparse.shape

(82797, 48371)

In [17]:
type(X_sparse)

scipy.sparse.csr.csr_matrix

**<font color='red'>Вопрос 1. </font> Выведите размерности матриц *X_train_sparse* и *X_test_sparse* – 4 числа на одной строке через пробел: число строк и столбцов матрицы *X_train_sparse*, затем число строк и столбцов матрицы *X_test_sparse*.**

In [483]:
X_train_sparse.shape[0], X_train_sparse.shape[1], X_test_sparse.shape[0], X_test_sparse.shape[1]

(253561, 48371, 82797, 48371)

**Сохраним в pickle-файлы объекты *X_train_sparse*, *X_test_sparse* и *y* (последний – в файл *kaggle_data/train_target.pkl*).**

In [19]:
with open(os.path.join(PATH_TO_DATA, 'X_train_sparse.pkl'), 'wb') as X_train_sparse_pkl:
    pickle.dump(X_train_sparse, X_train_sparse_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'X_test_sparse.pkl'), 'wb') as X_test_sparse_pkl:
    pickle.dump(X_test_sparse, X_test_sparse_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'train_target.pkl'), 'wb') as train_target_pkl:
    pickle.dump(y, train_target_pkl, protocol=2)

**Разобьем обучающую выборку на 2 части в пропорции 7/3, причем не перемешивая. Исходные данные упорядочены по времени, тестовая выборка по времени четко отделена от обучающей, это же соблюдем и здесь.**

In [484]:
train_share = int(.7 * X_train_sparse.shape[0])
X_train, y_train = X_train_sparse[:train_share, :], y[:train_share]
X_valid, y_valid  = X_train_sparse[train_share:, :], y[train_share:]

In [21]:
train_share, X_train.shape, X_valid.shape

(177492, (177492, 48371), (76069, 48371))

**Создайте объект `sklearn.linear_model.SGDClassifier` с логистической функцией потерь и параметром *random_state*=17. Остальные параметры оставьте по умолчанию, разве что *n_jobs*=-1 никогда не помешает. Обучите  модель на выборке `(X_train, y_train)`.**

In [485]:
sgd_logit = SGDClassifier(loss = 'log', random_state=17,  n_jobs=-1)#''' ВАШ КОД ЗДЕСЬ '''
sgd_logit.fit(X_train, y_train) #''' ВАШ КОД ЗДЕСЬ '''

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=17, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

**Сделайте прогноз в виде предсказанных вероятностей того, что это сессия Элис, на отложенной выборке *(X_valid, y_valid)*.**

In [486]:
logit_valid_pred_proba = sgd_logit.predict_proba(X_valid) #''' ВАШ КОД ЗДЕСЬ '''

**<font color='red'>Вопрос 2. </font> Посчитайте ROC AUC логистической регрессии, обученной с помощью стохастического градиентного спуска, на отложенной выборке. Округлите до 3 знаков после разделителя.**

In [487]:
ras = roc_auc_score(y_valid, logit_valid_pred_proba[:, 1])
print(ras)
#''' ВАШ КОД ЗДЕСЬ '''

0.9338469992188017


**<font color='red'>Вопрос 3-4 </font> Бенчмарки (оба) побиты. Тк соревнований два, то уточню, что искать надо в этом https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard мой ник Natalie, пока что это место 1335 с точностью 0.93008. Список мучений с подбором параметров описан в конце этого ноутбука.**

**Сделайте прогноз в виде предсказанных вероятностей отнесения к классу 1 для тестовой выборки с помощью той же *sgd_logit*, обученной уже на всей обучающей выборке (а не на 70%).**

In [25]:
%%time
sgd_logit.fit(X_train_sparse, y)#''' ВАШ КОД ЗДЕСЬ '''
logit_test_pred_proba = sgd_logit.predict_proba(X_test_sparse)[:, 1]#''' ВАШ КОД ЗДЕСЬ '''

Wall time: 620 ms


**Запишите ответы в файл и сделайте посылку на Kaggle. Дайте своей команде (из одного человека) на Kaggle говорящее название – по шаблону "[YDF & MIPT] Coursera_Username", чтоб можно было легко идентифицировать Вашу посылку на [лидерборде](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard/public).**

**Результат, который мы только что получили, соответствует бейзлайну "SGDCLassifer" на лидерборде, задача на эту неделю – как минимум его побить.**

In [26]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, len(predicted_labels) + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [27]:
write_to_submission_file(logit_test_pred_proba, 'kaggle_comp.csv')

### эксперимент с подбором параметров классификатора

In [28]:
test_sgd_logit = SGDClassifier(loss = 'log', random_state=17,  n_jobs=-1)
params = { 'tol': [0.000001, 0.00001]}
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)
grid_searcher = GridSearchCV(test_sgd_logit, params, cv = skf)
grid_searcher.fit(X_train, y_train)
pred_proba = grid_searcher.best_estimator_.predict_proba(X_valid)
ras = roc_auc_score(y_valid, pred_proba[:, 1])
print(ras)
print(grid_searcher.best_params_)

0.9403227713702091
{'tol': 1e-05}


In [29]:
X_train.shape, X_valid.shape

((177492, 48371), (76069, 48371))

In [30]:
grid_searcher.best_estimator_.fit(X_train_sparse, y)#''' ВАШ КОД ЗДЕСЬ '''
kaggle_pred_proba = grid_searcher.best_estimator_.predict_proba(X_test_sparse)[:, 1]
write_to_submission_file(kaggle_pred_proba, 'kaggle_comp_try1_alpha.csv')

In [414]:
#train_df[train_df['target'] == 1].shape, train_df[train_df['target'] == 0].shape
train_df_Alice = train_df[train_df['target'] == 1]
train_df_not_Alice = train_df[train_df['target'] == 0]
to_add_count = len(train_df_not_Alice) - len(train_df_Alice)
#balanced_df = train_df
balanced_ar = [] 
balanced_ar.append(list(train_df.values.flatten()))
balanced_ar = balanced_ar[0]

np.random.seed(10)
for i in range(0, to_add_count):
    random_index = np.random.choice(train_df_Alice.index, 1)
    row_to_add = train_df_Alice.loc[random_index[0]]
    #balanced_ar = np.hstack((balanced_ar, row_to_add))
    for val in row_to_add:
        balanced_ar.append(val)
cols = train_df.columns
balanced_ar = np.array(balanced_ar).reshape(train_df.shape[0] + to_add_count, train_df.shape[1])
#balanced_data = pd.DataFrame(columns = cols)
balanced_data = pd.DataFrame(data = balanced_ar, columns = cols)
#balanced_data = balanced_data.set_index('session_id')


In [212]:
balanced_data.to_csv('balanced_data.csv', index = False)
#balanced_data = pd.read_csv()

In [429]:
balanced_data['time1'] = pd.to_datetime(balanced_data['time1'])
balanced_data = balanced_data.sort_values(by = 'time1')

In [430]:
balanced_target = balanced_data.iloc[:, -1]
balanced_train_test_df = pd.concat([balanced_data.iloc[:, :-1], test_df])
balanced_train_test_df = balanced_train_test_df.replace('nan', 0)
balanced_train_test_df_balanced_sites = balanced_train_test_df[['site%d' % i for i in range(1, 11)]].fillna(0).astype('float').astype('int')

In [431]:
balanced_train_test_df_balanced_sites[:10]

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
21668,56,55,0,0,0,0,0,0,0,0
54842,56,55,56,55,0,0,0,0,0,0
77291,946,946,951,946,946,945,948,784,949,946
114020,945,948,949,948,945,946,947,945,946,946
146669,947,950,948,947,950,952,946,951,946,947
242170,952,947,953,946,947,946,953,955,946,947
57156,953,947,946,953,955,947,953,946,953,1033
240200,946,947,954,953,946,954,946,956,957,956
210685,946,956,946,946,955,954,946,946,946,948
98803,948,946,948,784,49,53,812,982,52,52


### Вводим новые признаки

In [637]:
train_test_df_to_sparse = train_test_df_sites#balanced_train_test_df_balanced_sites
times = train_df[['time' + str(i) for i in range(1, 11)]]#balanced_data[['time' + str(i) for i in range(1, 11)]]
times_test = test_df[['time' + str(i) for i in range(1, 11)]]
times = times.fillna(0)
times_test = times_test.fillna(0)
times = times.replace('nan', 0)
for col in times.columns:
    times[col] = times[col].astype('datetime64[ns]')
    times_test[col] = times_test[col].astype('datetime64[ns]')
train_test_times = pd.concat([times, times_test])
train_test_df_to_sparse = pd.concat([train_test_df_to_sparse.reset_index(drop=True), train_test_times.reset_index(drop=True)], axis = 1)
#train_test_df_to_sprse[10: len(times)] = times

In [638]:
train_test_df_to_sparse.shape

(336358, 20)

In [639]:
s = X_sparse.sum(axis = 0)
s_arg = s.argsort(axis = 1)
top_10_sites_codes = s_arg[:, -10:]
top_10_freqs = s[0, s_arg[:, -10:]]
#print(s)
#print(s_arg)
#print(top_10_sites_codes)
#print(top_10_freqs)
#print(s_arg.max())
#sites_names = [d_k for i in top_10_sites_codes.tolist()[0] for d_k, d_v in train_data_10users[1].items() if d_v[0] == i + 1]
#print(sites_names)

In [651]:
def feature_engineering(sparse_matrix, features, top_sites_codes, session_length = 10):
    gen_data_ar = []  
    cols = ['session_id']
    for c in features:
        cols.append(c)
    gen_data = pd.DataFrame(columns = cols)
    
    for row in sparse_matrix.itertuples():      
        chunck_ar = np.zeros(len(features) + 1 , dtype = float)
        #return row
        chunck_ar[0] = row.Index
        sessioin_sites_ids = row[1:session_length + 1]
        chunck_ar[1:len(sessioin_sites_ids)+1] = sessioin_sites_ids
        timestamps = row[11:]                
        #time_diff = [30 * 60 if abs((timestamps[i + 1] - timestamps[i]).total_seconds()) > 30 * 60 else (timestamps[i + 1] - timestamps[i]).total_seconds() for i, t in enumerate(timestamps) if i != len(timestamps) - 1]
        # время, проведенное на каждом сайте
        #chunck_ar[session_length + 1 : session_length + 1 + len(time_diff)] = time_diff                
        # время сессии
        #chunck_ar[session_length + 1] = 30 * 60 if abs((max(timestamps) - min(timestamps)).total_seconds()) > 30 * 60 else abs(max(timestamps) - min(timestamps)).total_seconds()#'''session_length'''
        # количество уникальных сайтов
        #chunck_ar[session_length + 1] = len(list(set(sessioin_sites_ids)))#'''1 + session_length'''
        chunck_ar[1 + session_length] = pd.to_datetime(timestamps[0]).hour#'''2 + session_length'''        
        chunck_ar[2 + session_length] = pd.to_datetime(timestamps[0]).dayofweek#'''3 + session_length'''
        chunck_ar[3 + session_length] = pd.to_datetime(timestamps[0]).dayofweek#'''2 + session_length'''
        #частота посещения топ-10 сайтов за сессию
        #chunck_ar[session_length + 3] = len([i for i in sessioin_sites_ids if i in top_sites_codes]) / session_length
        #время, проведенное на каком-то топовом сайте
        top_time = [30 * 60 if abs((timestamps[i + 1] - timestamps[i]).total_seconds()) > 30 * 60 else (timestamps[i + 1] - timestamps[i]).total_seconds() for i, t in enumerate(timestamps) if i != len(timestamps) - 1 and sessioin_sites_ids[i] in top_sites_codes]
        #среднее время, проведенное на топ-10 сайтов
        #time_diff_top = 0
        #if (len(top_time) != 0):
            #time_diff_top = sum(top_time) / len(top_time)             
        #chunck_ar[session_length + 3] = time_diff_top #session_length + 5 + session_length             
        # признак посещения каждого из топовых сайтов
        #chunck_ar[session_length + 4 : session_length + 4 + 11] = [1 if i in sessioin_sites_ids else 0 for i in features[-10:]] #session_length + 6 + session_length : session_length + 6 + session_length + 10       
        gen_data_ar.append(chunck_ar)#np.append(gen_data_ar, chunck_ar)
    gen_data_ar = np.array(gen_data_ar).reshape(sparse_matrix.shape[0], len(features) + 1)
    gen_data = pd.DataFrame(data = gen_data_ar, columns = cols)
    gen_data = gen_data.set_index('session_id')
    return gen_data

In [652]:
a = list(map(lambda i: i, top_10_sites_codes))[0].A[0]
b = [str(i) for i in a]
#['time_diff' + str(j) for j in range(1,10)] + \
feature_names_new = ['site' + str(i) for i in range(1,11)] + ['start_hour', 'day_of_week', 'day_scaled'] #+ b
                #['session_timespan', '#unique_sites', 'start_hour', 
                 #'day_of_week']#, 'top_10_freq', 'top_10_time'] + \
        #b
data = feature_engineering(train_test_df_to_sparse, feature_names_new, top_10_sites_codes)
#data_dummies = pd.get_dummies(data, columns=['day_of_week'])

In [662]:
data_dummies = pd.get_dummies(data, columns=['day_of_week'])

In [663]:
#data_dummies['day_scaled'] = StandardScaler().fit_transform(data_dummies['day_scaled'].values.reshape(-1, 1))

In [664]:
#data_dummies = data_dummies.drop(['day_of_week_0.0', 'day_of_week_1.0', 'day_of_week_2.0', 'day_of_week_3.0', 'day_of_week_4.0', 'day_of_week_5.0', ], axis = 1)

In [665]:
data_dummies.columns

Index([u'site1', u'site2', u'site3', u'site4', u'site5', u'site6', u'site7',
       u'site8', u'site9', u'site10', u'start_hour', u'day_scaled',
       u'day_of_week_0.0', u'day_of_week_1.0', u'day_of_week_2.0',
       u'day_of_week_3.0', u'day_of_week_4.0', u'day_of_week_5.0',
       u'day_of_week_6.0'],
      dtype='object')

In [666]:
data_without_user_id = data_dummies.iloc[:, 0:10]
flatten = data_without_user_id.values.flatten()
data_csr = [1] * len(flatten)
indptr_csr = np.arange(0, len(flatten) + 1, 10)
X_sparse_fe = csr_matrix((data_csr, flatten, indptr_csr))[:, 1:]
y_sparse_fe = train_test_df['target']#balanced_target

#train_test_sparse_fe = X_sparse_fe#''' ВАШ КОД ЗДЕСЬ '''
X_sparse_fe = hstack((X_sparse_fe, data_dummies.iloc[:, 10:])).tocsr()
X_train_sparse_fe = X_sparse_fe[:-82797]#''' ВАШ КОД ЗДЕСЬ '''
X_test_sparse_fe = X_sparse_fe[-82797:]#''' ВАШ КОД ЗДЕСЬ '''
y_fe = y_sparse_fe[:-82797]#y_sparse_fe

In [667]:
y_fe.shape, X_train_sparse_fe.shape

((253561L,), (253561, 48380))

In [668]:
data.shape, data_without_user_id.shape, train_test_df_sites.shape, X_sparse_fe.shape, X_sparse.shape

((336358, 13), (336358, 10), (336358, 10), (336358L, 48380L), (336358, 48371))

In [669]:
#data_train_sparse = data.iloc[:253561]
#data_test_sparse = data.iloc[253561:]
#data_y = y_sparse[:253561]

train_share_fe = int(.7 * X_train_sparse_fe.shape[0])
X_train_fe, y_train_fe = X_train_sparse_fe[:train_share_fe, :], y_fe[:train_share_fe]
X_valid_fe, y_valid_fe  = X_train_sparse_fe[train_share_fe:, :], y_fe[train_share_fe:]

In [488]:


data_without_user_id = train_test_df_sites.iloc[:, 0:10]
flatten = data_without_user_id.values.flatten()
data_csr = [1] * len(flatten)
indptr_csr = np.arange(0, len(flatten) + 1, 10)
X_sparse = csr_matrix((data_csr, flatten, indptr_csr))[:, 1:]
y_sparse = train_test_df['target']

train_test_sparse = X_sparse#''' ВАШ КОД ЗДЕСЬ '''
X_train_sparse = X_sparse[:253561]#''' ВАШ КОД ЗДЕСЬ '''
X_test_sparse = X_sparse[253561:]#''' ВАШ КОД ЗДЕСЬ '''
y = y_sparse[:253561]#''' ВАШ КОД ЗДЕСЬ '''

train_share = int(.7 * X_train_sparse.shape[0])
X_train, y_train = X_train_sparse[:train_share, :], y[:train_share]
X_valid, y_valid  = X_train_sparse[train_share:, :], y[train_share:]

test_sgd_logit = SGDClassifier(loss = 'log', random_state=17,  n_jobs=-1)
params = { }
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)
grid_searcher = GridSearchCV(test_sgd_logit, params, cv = skf)
grid_searcher.fit(X_train, y_train)
pred_proba = grid_searcher.best_estimator_.predict_proba(X_valid)
ras = roc_auc_score(y_valid, pred_proba[:, 1])
print(ras)
print(grid_searcher.best_params_)

0.9338469992188017
{}


In [479]:
X_train.shape

(177492, 48371)

In [670]:
test_sgd_logit = SGDClassifier(loss = 'log', random_state=17,  n_jobs=-1)
params = { }
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)
grid_searcher = GridSearchCV(test_sgd_logit, params, cv = skf)
grid_searcher.fit(X_train_fe, y_train_fe)
pred_proba = grid_searcher.best_estimator_.predict_proba(X_valid_fe)
ras = roc_auc_score(y_valid_fe, pred_proba[:, 1])
print(ras)
print(grid_searcher.best_params_)

0.9561715250071299
{}


сбалансированные классы: <br>
 только сайты: 8788 <br>
 сайты + время сессии: 8065 <br>
 сайты + количество уникальных сайтов: 8784 <br>
 сайты + час начала сессии: 9226 <br>
 сайты + час начала сессии + день начала сессии (dummies) : 9319 <br>
 сайты + час начала сессии + день начала сессии (dummies) + частота посещения топ-10 сайтов : 9318 <br>
 сайты + час начала сессии + день начала сессии (dummies) + время, проведенное на каждом сайте : 7733<br>
 сайты + час начала сессии + день начала сессии (dummies) + признак посещений каждого из топовых сайтов: 9319<br>
сайты + час начала сессии + день начала сессии (dummies) + ср.время, проведенное на топ-10 сайтов: 892 <br>
сайты + час начала сессии (масштабированный) + день начала сессии (dummies) : 9303 <br>
сайты + час начала сессии (масштабированный) + день начала сессии (dummies) только среда: 9343 <br>
сайты + час начала сессии + день начала сессии (dummies) только среда: 9361 (ухудшение в kaggle) <br>
сайты + час начала сессии + день начала сессии (dummies) только среда и воскресенье: 9372 (ухудшение в kaggle)<br>
сайты + час начала сессии (масштабированный) + день начала сессии (dummies) только среда и воскресенье: 9353 (ухудшение в kaggle)<br>
сайты + час начала сессии (не и масштабированный) + день начала сессии (dummies) пн, вт, вс + месяца (кр. 2, 3, 4, 11, 12): 9323

несбалансированные классы:<br>
сайты + час начала сессии + день начала сессии (dummies) + день начала сессии : 0.9561 (это сработало и помогло побить указанные бенчмарки)

In [585]:
X_train_sparse_fe[:15].todense().shape

(15L, 48374L)

In [671]:
sgd_logit.fit(X_train_sparse_fe, y_fe)#''' ВАШ КОД ЗДЕСЬ '''
pred_proba_fe = sgd_logit.predict_proba(X_test_sparse_fe)[:, 1]#''' ВАШ КОД ЗДЕСЬ '''
write_to_submission_file(pred_proba_fe, 'kaggle_comp_new_fe.csv')

## Критерии оценки работы (только для Peer Review в специализации):
- Правильные ли получились размерности матриц в п. 1? (max. 2 балла)
- Правильным ли получилось значения ROC AUC в п. 2? (max. 4 балла)
- Побит ли бенчмарк "sgd_logit_benchmark.csv" на публичной части рейтинга в соревновании Kaggle? (max. 2 балла)
- Побит ли бенчмарк "Logit +3 features" на публичной части рейтинга в соревновании Kaggle? (max. 2 балла)

In [189]:
y_valid.shape

(76068,)

In [190]:
pred_proba[:, 1].shape

(76069,)

## Пути улучшения
На этой неделе дается много времени на соревнование. Не забывайте вносить хорошие идеи, к которым Вы пришли по ходу соревнования, в описание финального проекта (`html`, `pdf` или `ipynb`). Это только в случае, если вы проходите специализацию.
Что можно попробовать:
 - Использовать ранее построенные признаки для улучшения модели (проверить их можно на меньшей выборке по 150 пользователям, отделив одного из пользователей от остальных – это быстрее)
 - Настроить параметры моделей (например, коэффициенты регуляризации)
 - Если позволяют мощности (или хватает терпения), можно попробовать смешивание (блендинг) ответов бустинга и линейной модели. [Вот](http://mlwave.com/kaggle-ensembling-guide/) один из самых известных тьюториалов по смешиванию ответов алгоритмов, также хороша [статья](https://alexanderdyakonov.wordpress.com/2017/03/10/cтекинг-stacking-и-блендинг-blending) Александра Дьяконова
 - Обратите внимание, что в соревновании также даны исходные данные о посещенных веб-страницах Элис и остальными 1557 пользователями (*train.zip*). По этим данным можно сформировать свою обучающую выборку. 

На 6 неделе мы пройдем большой тьюториал по Vowpal Wabbit и попробуем его в деле, на данных соревнования.